In [1]:
# read T5 scores for each seg
import collections
import math
from tqdm import tqdm
#score is dictionary of qid, docsegid, t5score
scores = collections.defaultdict(dict)
with open('./d2qseg.dev.tsv') as f_query_run_ids, open(
        './mono_preds_d2qseg.dev.t5') as f_pred:
    for line_query_passage_id, line_pred in tqdm(zip(f_query_run_ids, f_pred)):
        query_id, passage_id, _ = line_query_passage_id.strip().split('\t')
        docseg_id = passage_id.split('.')[0]
        _, score = line_pred.strip().split('\t')
        score = float(score)
        scores[query_id][docseg_id] = score

51893214it [01:51, 466495.70it/s]


In [ ]:
# get ltr top k
from collections import defaultdict
counts = defaultdict(int)
with open('run.ltr.doc.300from3000.tsv', 'w') as output_f:
    with open('../run.ltr.doc.3000.tsv') as input_f:
        for line in tqdm(input_f):
            cols = line.split('\t')
            qid = cols[0]
            if(counts[qid] < 300):
                docid = cols[2]
                counts[qid] += 1
                output_f.write(f'{qid}\t{docid}\t{counts[qid]}\n')

15573204it [00:16, 949544.45it/s] 


In [ ]:
#get T5 for each seg for ltr topk and take maxP for each doc and reranked 
examples = None
examples = collections.defaultdict(dict)
with open('./run.ltr.doc.300from3000.tsv') as fcan, open('./run.monoT5.doc.ltr.300from3000.tsv', 'w') as fout:
    for line_query_passage_id in tqdm(fcan):
        query_id, passage_id, _= line_query_passage_id.strip().split('\t')
        docseg_id = passage_id.split('.')[0]
        doc_id = docseg_id.split('#')[0]
        score = scores[query_id][docseg_id]
        score = float(score)
        if doc_id not in examples[query_id]:
            examples[query_id][doc_id] = [score]
        else:
            examples[query_id][doc_id].append(score)
        #examples is qid, doc_id with array of scores
    
    for query_id, doc_ids_scores in tqdm(examples.items()):
        #by taking maxP to get doc score
        doc_ids_scores = [
            (doc_id, max(scores))
            for doc_id, scores in doc_ids_scores.items()]

        doc_ids_scores.sort(key=lambda x: x[1], reverse=True)
        rank = 1
        for doc_id, score in doc_ids_scores:
            score = math.exp(score)
            # Adds small score based on rank to avoid duplicated scores.
            score += 1e-6 / rank
            fout.write(f'{query_id}\t{doc_id}\t{rank}\n')
            rank += 1

1557900it [00:04, 360193.77it/s]
100%|██████████| 5193/5193 [00:01<00:00, 4676.70it/s]


In [ ]:
#only takes top 100 for each doc for msmarco_doc_eval.py
with open ('./run.monoT5.doc.ltr.200from3000.tsv', 'w') as fout:
    for query_id, doc_ids_scores in tqdm(examples.items()):
            doc_ids_scores = [
                (doc_id, max(scores))
                for doc_id, scores in doc_ids_scores.items()]

            doc_ids_scores.sort(key=lambda x: x[1], reverse=True)
            rank = 1
            for doc_id, score in doc_ids_scores:
                if (rank < 100):
                    score = math.exp(score)
            # Adds small score based on rank to avoid duplicated scores.
                    score += 1e-6 / rank
                    fout.write(f'{query_id}\t{doc_id}\t{rank}\n')
                rank += 1

100%|██████████| 5193/5193 [00:00<00:00, 6992.82it/s]


In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/msmarco_doc_eval.py \
 --judgments ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt --run ./run.monoT5.doc.ltr.300from3000.tsv